In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import re

In [ ]:
train_path = '/content/drive/MyDrive/Datasets/tweets.txt'
#dev_path = '/content/drive/MyDrive/mediaeval2016/dev-ekphrasis.csv'
test_path = '/content/drive/MyDrive/Datasets/tweets-test.txt'

In [ ]:
train = pd.read_csv(train_path, sep = '\t', header = 0)
train.info()
display(train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14277 entries, 0 to 14276
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetId     14277 non-null  int64 
 1   tweetText   14277 non-null  object
 2   userId      14277 non-null  int64 
 3   imageId(s)  14277 non-null  object
 4   username    14277 non-null  object
 5   timestamp   14277 non-null  object
 6   label       14277 non-null  object
dtypes: int64(2), object(5)
memory usage: 780.9+ KB


,tweetId,tweetText,userId,imageId(s),username,timestamp,label
0,263046056240115712,¿Se acuerdan de la película: “El día después d...,21226711,sandyA_fake_46,iAnnieM,Mon Oct 29 22:34:01 +0000 2012,fake
1,262995061304852481,@milenagimon: Miren a Sandy en NY! Tremenda i...,192378571,sandyA_fake_09,CarlosVerareal,Mon Oct 29 19:11:23 +0000 2012,fake
2,262979898002534400,"Buena la foto del Huracán Sandy, me recuerda a...",132303095,sandyA_fake_09,LucasPalape,Mon Oct 29 18:11:08 +0000 2012,fake
3,262996108400271360,Scary shit #hurricane #NY http://t.co/e4JLBUfH,241995902,sandyA_fake_29,Haaaaarryyy,Mon Oct 29 19:15:33 +0000 2012,fake
4,263018881839411200,My fave place in the world #nyc #hurricane #sa...,250315890,sandyA_fake_15,princess__natt,Mon Oct 29 20:46:02 +0000 2012,fake


In [ ]:
test = pd.read_csv(test_path, sep='\t', header=0)
test.info()
display(test.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweetId     3755 non-null   int64 
 1   tweetText   3755 non-null   object
 2   userId      3755 non-null   int64 
 3   imageId(s)  3755 non-null   object
 4   username    3755 non-null   object
 5   timestamp   3755 non-null   object
 6   label       3755 non-null   object
dtypes: int64(2), object(5)
memory usage: 205.5+ KB


,tweetId,tweetText,userId,imageId(s),username,timestamp,label
0,578854927457349632,kereeen RT @Shyman33: Eclipse from ISS.... htt...,70824972,eclipse_01,peay_s,Fri Mar 20 09:45:43 +0000 2015,fake
1,578874632670953472,Absolutely beautiful! RT @Shyman33: Eclipse fr...,344707006,eclipse_01,JaredUcanChange,Fri Mar 20 11:04:02 +0000 2015,fake
2,578891261353984000,“@Shyman33: Eclipse from ISS.... http://t.co/C...,224839607,eclipse_01,tpjp1231,Fri Mar 20 12:10:06 +0000 2015,fake
3,578846612312748032,Eclipse from ISS.... http://t.co/En87OtvsU6,134543073,eclipse_01,Shyman33,Fri Mar 20 09:12:41 +0000 2015,fake
4,578975333841551360,@ebonfigli: Éclipse vue de l'ISS... Autre chos...,1150728872,eclipse_01,Epimethee_,Fri Mar 20 17:44:11 +0000 2015,fake


# Pre-processing

In [ ]:
!pip install ekphrasis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 8.2 MB/s eta 0:00:00


In [ ]:
!pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import preprocessor as p
from googletrans import Translator

In [ ]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'user'],
    fix_html=True,  # fix HTML tokens

    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=False).tokenize,

    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons],
    mode='fast'
)


/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))
/usr/local/lib/python3.10/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [ ]:
!pip install deep-translator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from deep_translator import GoogleTranslator
def preprocess_text(dataset):
    p.set_options(p.OPT.RESERVED)
    dataset['CleanText'] = dataset['tweetText'].apply(lambda text: p.clean(text))
    dataset['CleanText'] = dataset['tweetText'].apply(lambda text: re.sub(r'\n', ' ', text))
    dataset['CleanText'] = dataset['tweetText'].apply(lambda text: re.sub(r'\\n', ' ', text))
    dataset['CleanText'] = dataset['tweetText'].apply(lambda text: re.sub(r'(&amp;|&gt;|&lt;)', " ", text))
    dataset['CleanText'] = dataset['tweetText'].apply(lambda text: " ".join(text_processor.pre_process_doc(text)))
    dataset['CleanText'] = dataset['tweetText'].apply(lambda text: re.sub(r"\s+", " ", text))
    #dataset['CleanText'] = dataset['tweetText'].apply(lambda text: GoogleTranslator(source='auto', target='en').translate(text))

In [ ]:
def preprocessing(text):
  p.set_options(p.OPT.RESERVED) #Delete RT
  text = p.clean(text)

  text = re.sub(r'\n', ' ', text)
  text = re.sub(r'\\n', ' ', text)
  text = re.sub(r'(&amp;|&gt;|&lt;)', " ", text) #Delete some special words
  text = " ".join(text_processor.pre_process_doc(text))
  text = re.sub(r"\s+", " ", text)

  return text

In [ ]:
#preprocess_text(train)
preprocess_text(test)

In [ ]:
preprocess_text(train)

In [ ]:
import tqdm

# train['cleaned_text'] = train['post_text']
# dev['cleaned_text'] = dev['post_text']
# test['cleaned_text'] = test['post_text']
translator = GoogleTranslator(source='auto', target='en')
for i in tqdm.tqdm(range(len(train['tweetText']))):
  sent = train['CleanText'][i]
  # sent = preprocessing(sent)
  try:
      train['CleanText'][i] = translator.translate(sent)
  except:
      train['CleanText'][i] = sent

for i in tqdm.tqdm(range(len(test['tweetText']))):
  sent = test['CleanText'][i]
  # sent = preprocessing(sent)
  test['CleanText'][i] = translator.translate(sent)

  0%|          | 0/14277 [00:00<?, ?it/s]<ipython-input-24-3fcdfb36851f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['CleanText'][i] = sent
 21%|██        | 2974/14277 [19:03<1:02:25,  3.02it/s]<ipython-input-24-3fcdfb36851f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['CleanText'][i] = translator.translate(sent)
  0%|          | 0/3755 [00:00<?, ?it/s]<ipython-input-24-3fcdfb36851f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [ ]:
train['enc_label'] = train['label']
train['enc_label'] = train['enc_label'].apply(lambda x: 1 if x=='fake' or x=='humor' else 0)
test['enc_label'] = test['label']
test['enc_label'] = test['enc_label'].apply(lambda x: 1 if x=='fake' or x=='humor'  else 0)

In [ ]:
display(train.head()), display(test.head())

,tweetId,tweetText,userId,imageId(s),username,timestamp,label,CleanText,enc_label
0,263046056240115712,¿Se acuerdan de la película: “El día después d...,21226711,sandyA_fake_46,iAnnieM,Mon Oct 29 22:34:01 +0000 2012,fake,Do you remember the movie: “The day after tomo...,1
1,262995061304852481,@milenagimon: Miren a Sandy en NY! Tremenda i...,192378571,sandyA_fake_09,CarlosVerareal,Mon Oct 29 19:11:23 +0000 2012,fake,@milenagimon: Look at Sandy in NY! Tremendous ...,1
2,262979898002534400,"Buena la foto del Huracán Sandy, me recuerda a...",132303095,sandyA_fake_09,LucasPalape,Mon Oct 29 18:11:08 +0000 2012,fake,"Good photo of Hurricane Sandy, it reminds me o...",1
3,262996108400271360,Scary shit #hurricane #NY http://t.co/e4JLBUfH,241995902,sandyA_fake_29,Haaaaarryyy,Mon Oct 29 19:15:33 +0000 2012,fake,Scary shit #hurricane #NY http://t.co/e4JLBUfH,1
4,263018881839411200,My fave place in the world #nyc #hurricane #sa...,250315890,sandyA_fake_15,princess__natt,Mon Oct 29 20:46:02 +0000 2012,fake,My fave place in the world #nyc #hurricane #sa...,1


,tweetId,tweetText,userId,imageId(s),username,timestamp,label,CleanText,enc_label
0,578854927457349632,kereeen RT @Shyman33: Eclipse from ISS.... htt...,70824972,eclipse_01,peay_s,Fri Mar 20 09:45:43 +0000 2015,fake,kereeen RT @Shyman33: Eclipse from ISS.... htt...,1
1,578874632670953472,Absolutely beautiful! RT @Shyman33: Eclipse fr...,344707006,eclipse_01,JaredUcanChange,Fri Mar 20 11:04:02 +0000 2015,fake,Absolutely beautiful! RT @Shyman33: Eclipse fr...,1
2,578891261353984000,“@Shyman33: Eclipse from ISS.... http://t.co/C...,224839607,eclipse_01,tpjp1231,Fri Mar 20 12:10:06 +0000 2015,fake,“@Shyman33: Eclipse from ISS.... http://t.co/C...,1
3,578846612312748032,Eclipse from ISS.... http://t.co/En87OtvsU6,134543073,eclipse_01,Shyman33,Fri Mar 20 09:12:41 +0000 2015,fake,Eclipse from ISS.... http://t.co/En87OtvsU6,1
4,578975333841551360,@ebonfigli: Éclipse vue de l'ISS... Autre chos...,1150728872,eclipse_01,Epimethee_,Fri Mar 20 17:44:11 +0000 2015,fake,@ebonfigli: Eclipse seen from the ISS... Anoth...,1


(None, None)

In [ ]:
train.to_csv('/content/drive/MyDrive/Datasets/train-ekphrasis.csv',index=False)
test.to_csv('/content/drive/MyDrive/Datasets/test-ekphrasis.csv',index=False)